In [293]:
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt

In [294]:
# Download the data from yfinance

df = yf.download('INFY.NS', '2000-01-01', '2022-12-31')
df = round(df,2)
df.index = df.index.strftime("%Y-%m-%d")

[*********************100%***********************]  1 of 1 completed


In [295]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,122.07,122.07,122.07,122.07,80.90,657536
2000-01-04,131.25,131.84,126.98,131.69,87.28,7191808
2000-01-05,122.66,126.95,121.15,121.15,80.29,21069440
2000-01-06,119.19,119.53,111.46,111.46,73.87,10495616
2000-01-07,102.54,102.54,102.54,102.54,67.96,971392
...,...,...,...,...,...,...
2022-12-26,1500.50,1509.90,1496.35,1502.40,1462.79,4115459
2022-12-27,1509.95,1521.45,1497.50,1514.85,1474.92,4860076
2022-12-28,1505.10,1524.95,1498.50,1510.15,1470.34,5029860


In [296]:
# rename the columns

df = df.reset_index()

df.columns = ('date','open', 'high', 'low', 'close', 'adj_close', 'volume')
df

,date,open,high,low,close,adj_close,volume
0,2000-01-03,122.07,122.07,122.07,122.07,80.90,657536
1,2000-01-04,131.25,131.84,126.98,131.69,87.28,7191808
2,2000-01-05,122.66,126.95,121.15,121.15,80.29,21069440
3,2000-01-06,119.19,119.53,111.46,111.46,73.87,10495616
4,2000-01-07,102.54,102.54,102.54,102.54,67.96,971392
...,...,...,...,...,...,...,...
5743,2022-12-26,1500.50,1509.90,1496.35,1502.40,1462.79,4115459
5744,2022-12-27,1509.95,1521.45,1497.50,1514.85,1474.92,4860076
5745,2022-12-28,1505.10,1524.95,1498.50,1510.15,1470.34,5029860
5746,2022-12-29,1504.00,1521.30,1501.00,1517.55,1477.55,4624745


In [297]:
# compute simple moving average

n = 200

df['sma'] = round(df['adj_close'].rolling(window=n).mean(),2)
df = df.dropna().reset_index(drop = True)

In [298]:
df

,date,open,high,low,close,adj_close,volume,sma
0,2000-10-06,116.88,118.36,113.14,117.72,78.05,66243904,84.05
1,2000-10-09,117.19,117.36,114.86,115.53,76.60,32816768,84.03
2,2000-10-10,116.39,117.80,113.05,114.70,76.05,61546048,83.98
3,2000-10-11,110.94,111.70,106.02,106.48,70.59,64735168,83.93
4,2000-10-12,106.09,107.03,102.44,106.16,70.38,69716096,83.91
...,...,...,...,...,...,...,...,...
5544,2022-12-26,1500.50,1509.90,1496.35,1502.40,1462.79,4115459,1495.76
5545,2022-12-27,1509.95,1521.45,1497.50,1514.85,1474.92,4860076,1494.67
5546,2022-12-28,1505.10,1524.95,1498.50,1510.15,1470.34,5029860,1493.38
5547,2022-12-29,1504.00,1521.30,1501.00,1517.55,1477.55,4624745,1492.06


# Generating Signals

In [299]:
df['signal'] = ''

for i in range(len(df)):
    
    if (df['adj_close'].iloc[i-1] < df['sma'].iloc[i-1]) & (df['adj_close'].iloc[i] > df['sma'].iloc[i]):
        df['signal'].iloc[i] = 'buy'
    
    elif (df['adj_close'].iloc[i-1] > df['sma'].iloc[i]) & (df['adj_close'].iloc[i] < df['sma'].iloc[i]):
        df['signal'].iloc[i] = 'sell'
        
    else:
        df['signal'].iloc[i] = ''
        
# shift the signals to the next day as the trade will be taken the next day

df['signal'] = df['signal'].shift(1)

df    

,date,open,high,low,close,adj_close,volume,sma,signal
0,2000-10-06,116.88,118.36,113.14,117.72,78.05,66243904,84.05,NaN
1,2000-10-09,117.19,117.36,114.86,115.53,76.60,32816768,84.03,sell
2,2000-10-10,116.39,117.80,113.05,114.70,76.05,61546048,83.98,
3,2000-10-11,110.94,111.70,106.02,106.48,70.59,64735168,83.93,
4,2000-10-12,106.09,107.03,102.44,106.16,70.38,69716096,83.91,
...,...,...,...,...,...,...,...,...,...
5544,2022-12-26,1500.50,1509.90,1496.35,1502.40,1462.79,4115459,1495.76,
5545,2022-12-27,1509.95,1521.45,1497.50,1514.85,1474.92,4860076,1494.67,
5546,2022-12-28,1505.10,1524.95,1498.50,1510.15,1470.34,5029860,1493.38,
5547,2022-12-29,1504.00,1521.30,1501.00,1517.55,1477.55,4624745,1492.06,


# Generate Tradelog

In [300]:
# filtering just buy signals

buy_df = df.loc[df['signal']=='buy'][['date','adj_close','signal']].reset_index(drop = True)
buy_df.columns = ('entry_date', 'close', 'buy_signal')
buy_df

,entry_date,close,buy_signal
0,2001-11-27,39.31,buy
1,2002-03-04,37.95,buy
2,2002-04-05,37.78,buy
3,2002-09-12,38.07,buy
4,2002-09-18,37.27,buy
...,...,...,...
63,2020-06-19,652.37,buy
64,2020-06-24,660.33,buy
65,2020-06-29,676.60,buy
66,2022-11-02,1512.35,buy


In [301]:
# filtering just sell signals

sell_df = df.loc[df['signal']=='sell'][['date','adj_close','signal']].reset_index(drop = True)
sell_df.columns = ('exit_date', 'close', 'sell_signal')
sell_df

,exit_date,close,sell_signal
0,2000-10-09,76.60,sell
1,2002-03-01,37.63,sell
2,2002-04-02,37.07,sell
3,2002-05-29,37.02,sell
4,2002-09-13,37.29,sell
...,...,...,...
61,2020-06-22,650.66,sell
62,2020-06-26,691.81,sell
63,2022-04-19,1488.62,sell
64,2022-11-03,1490.40,sell


In [302]:
# adjust the tradelog incase if sell is trigerred before buy

if buy_df['entry_date'][0] > sell_df['exit_date'][0]:
    sell_df = sell_df[1:].reset_index(drop = True)
    
if len(buy_df) > len(sell_df):
    buy_df = buy_df[:-1].reset_index(drop = True)

In [303]:
# conact the both buy & sell signals accordingly

tradelog_df = pd.concat([buy_df, sell_df], axis=1)
tradelog_df

,entry_date,close,buy_signal,exit_date,close,sell_signal
0,2001-11-27,39.31,buy,2002-03-01,37.63,sell
1,2002-03-04,37.95,buy,2002-04-02,37.07,sell
2,2002-04-05,37.78,buy,2002-05-29,37.02,sell
3,2002-09-12,38.07,buy,2002-09-13,37.29,sell
4,2002-09-18,37.27,buy,2002-09-19,36.51,sell
...,...,...,...,...,...,...
62,2020-06-09,663.52,buy,2022-04-19,1488.62,sell
63,2020-06-19,652.37,buy,2022-11-03,1490.40,sell
64,2020-06-24,660.33,buy,2022-12-16,1482.07,sell
65,2020-06-29,676.60,buy,NaN,NaN,NaN
